Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [0]:
import pandas as pd
import httpimport
import re

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [0]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

In [0]:
def preprocess( document ):
    string = ""
    for line in document:
        line = re.sub( "\s", " ", line )
        line = re.sub( "_", "", line )
        line = re.sub( "[^\w\s]", "", line )
        string += line

    return string

def unk( string ):
    d = {}
    for c in string:
        if c not in d:
            d[c] = 0
        d[c] += 1
    replacement = []
    # replacing charecters appearing less than 5 times
    for ( k, v ) in d.items():
        if v <= 5:
            replacement.append( k )
    replacement = '[' + "".join( replacement ) + ']'
    return replacement

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.

In [0]:
from sklearn.model_selection import train_test_split
import sys
import nltk 
from nltk.corpus import udhr
from nltk import word_tokenize
from nltk import bigrams
from nltk.util import ngrams
from nltk import FreqDist
from nltk import ConditionalFreqDist
import itertools
from collections import Counter


xTrain, xTest = train_test_split(train, test_size = 0.2, random_state = 0)
train_sentences = []
for i in range(0,len(xTrain)):
  new = list(itertools.chain.from_iterable(xTrain[i][1:len(xTrain[i])]))
  check = ' '.join(new) + '/n'
  train_sentences.append(check)

sentence = ''
for i in range(0,len(train_sentences)):
  train_sentences[i] = train_sentences[i].replace(u'\xa0', u' ')
  sentence = sentence + ' ' + train_sentences[i]

train_sentences = [item.lower() for item in train_sentences]
sentence = sentence.lower()

test_sentences = []
for i in range(0,len(xTest)):
  new = list(itertools.chain.from_iterable(xTest[i][1:len(xTest[i])]))
  check = ' '.join(new) + '/n'
  test_sentences.append(check)

sentenceTest = ''
for i in range(0,len(test_sentences)):
  test_sentences[i] = test_sentences[i].replace(u'\xa0', u' ')
  sentenceTest = sentenceTest + ' ' + test_sentences[i]

sentenceTest = sentenceTest.lower()


testFinal = []
for i in range(0,len(test)):
  new = list(itertools.chain.from_iterable(test[i][1:len(test[i])]))
  check = ' '.join(new) + '/n'
  testFinal.append(check)

for i in range(0,len(testFinal)):
  testFinal[i] = testFinal[i].replace(u'\xa0', u' ')

In [0]:
def ngrams( string, replacement, n, d ):
    if len( replacement ) > 2:
        string = re.sub( replacement, "?", string )
    length = len( string )
    for i in range( 0, length - n + 1 ):
        k = string[i:i + n]
        if k not in d:
            d[k] = 0
        d[k] += 1

def LanguageModel( trainFiles):
    lm = {"unigram": {"c": {}, "t": 0},
          "bigram" : {"c": {}, "t": 0},
          "trigram": {"c": {}, "t": 0}}
    ngram = ["unigram", "bigram", "trigram"]
    content = ""
    for document in trainFiles:
        content += preprocess( document)
    replacement = unk( content )
    for document in trainFiles:
        content = preprocess( document)
        ngrams( content, replacement, 1, lm["unigram"]["c"] )
        ngrams( content, replacement, 2, lm["bigram" ]["c"] )
        ngrams( content, replacement, 3, lm["trigram"]["c"] )
    for name in ngram:
        lm[name]["t"] = sum( lm[name]["c"].values() )
    return lm

def buildLanguageModel():
    ngram = ["unigram", "bigram", "trigram"]
    trainFiles = train_sentences
    heldOutFiles = test_sentences
    lm = LanguageModel( trainFiles)
    for name in ngram:
        with open( "./sample_data" + "/" + name, "w" ) as f:
            f.write( str( lm[name]["t"] ) + "\n" )
            for ( k, v ) in lm[name]["c"].items():
                f.write( k + " " + str( v ) + "\n" )

In [0]:
def part1_1():
    buildLanguageModel( )

In [0]:
part1_1()

In [0]:
def perplexity( content, lm, **kwargs ):
    length = len( content )
    log2p = 0
    if( length <= 2 ):
        raise Exception( "Too short content." )
    if "func" in kwargs:
        if kwargs["func"] == "Interplotation":
            for i in range( length - 1 ):
                p = interplotation( lm, kwargs["lambdas"], content[i:i + 3] )
                log2p += math.log2( p )
        elif kwargs["func"] == "AddK":
            for i in range( length - 1 ):
                p = addK( lm,content[i:i + 3] )
                log2p += math.log2( p )
        else:
            raise Exception( "Cannot find the smoothing function." )
    else:
        raise Exception( "No smoothing function." )
    log2p *= -1 / ( length - 2 )
    ppw = 2 ** log2p
    return ppw


## 1.2
Calculate the perplexity for each document in the test set using linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [0]:
def interplotation( lm, lambdas, s ):
    s1 = s[2:]
    s2 = s[1:]
    s3 = s[0:]
    if s1 not in lm["unigram"]["c"]:
        p1 = lm["unigram"]["c"]["?"] / lm["unigram"]["t"]
    else:
        p1 = lm["unigram"]["c"][s1] / lm["unigram"]["t"]
    if s2 not in lm["bigram"]["c"] or s1 not in lm["unigram"]["c"]:
        p2 = 0
    else:
        p2 = lm["bigram" ]["c"][s2] / lm["unigram" ]["c"][s1]
    if s3 not in lm["trigram"]["c"] or s2 not in lm["bigram"]["c"]:
        p3 = 0
    else:
        p3 = lm["trigram"]["c"][s3] / lm["bigram"]["c"][s2]
    p = lambdas[1] * p1 + lambdas[2] * p2 + lambdas[3] * p3
    return p

def gridSearch( lambdas, lm, heldOutFiles ):
    minAvg = float( "inf" )
    for lambd in lambdas:
        avg = 0
        for name in heldOutFiles:
            content = preprocess( name )
            avg += perplexity( content, lm, func = "Interplotation", lambdas = lambd )
        if( avg < minAvg ):
            minAvg = avg
            bestLambda = lambd
    return bestLambda

def interplotationSoomthing():
    # Get new language model
    #trainFiles, heldOutFiles = getFilesName( trainDataPath, ratio = ratio, shuffle = True )
    trainFiles = train_sentences
    heldOutFiles = test_sentences
    lm = LanguageModel( trainFiles)

    # Choose lambdas by grid search and perplexity
    lambdas = ( {1: x / 10, 2: y / 10, 3: ( 10 - x - y ) / 10}
                   for x in range( 1, 10, 1 ) for y in range( 1, 10 - x, 1 ) )

    lambdas = gridSearch( lambdas, lm, heldOutFiles )


    namePairs = {}
    #filesName, _ = getFilesName( testDataPath, suffix = "" )
    #for fileName in filesName:
    for i in range(0,len(testFinal)):
        content = preprocess( testFinal[i] )
        ppw = perplexity( content, lm, func = "Interplotation", lambdas = lambdas )
        namePairs[test_files[i]] = ppw
    fps = sorted( namePairs.items(), key = lambda x: x[1], reverse = True )

    for fp in fps:
      print("File NAME: "+fp[0]+"--")
      print("PErplexity :"+str(fp[1]))


In [0]:
def part1_2():
    interplotationSoomthing( )

In [76]:
part1_2()

File NAME: br94ag01.txt--
PErplexity :34.41193809411483
File NAME: br94ma01.txt--
PErplexity :33.42121016664539
File NAME: br94fe1.txt--
PErplexity :32.893170412773195
File NAME: ag94se06.txt--
PErplexity :32.45337119272047
File NAME: br94de01.txt--
PErplexity :32.002836333752406
File NAME: br94jl01.txt--
PErplexity :31.970463030481646
File NAME: br94ja04.txt--
PErplexity :31.096257485689367
File NAME: br94ju01.txt--
PErplexity :30.977913202786507
File NAME: ag94ag02.txt--
PErplexity :30.543432226288036
File NAME: ag94ju07.txt--
PErplexity :30.515977772604884
File NAME: ag94ou04.txt--
PErplexity :30.40852852743871
File NAME: br94ab02.txt--
PErplexity :30.25046572385523
File NAME: ag94ab12.txt--
PErplexity :30.171302096699208
File NAME: ag94ma03.txt--
PErplexity :29.9525925840757
File NAME: ag94de06.txt--
PErplexity :29.9467445364906
File NAME: ag94no01.txt--
PErplexity :29.10338092759246
File NAME: ag94jl12.txt--
PErplexity :28.986540437932113
File NAME: ag94mr1.txt--
PErplexity :27.49

Here the Threshold lies somewhere between 23.76346727493155 - 23.658837582907445 ; Any perplexity value above 23.76346727493155 (included) is non english value. perplexity value 23.658837582907445 (included) and below are english documents.

## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [0]:
def loadLanguageModel( loadPath ):
    lm = {}
    ngram = ["unigram", "bigram", "trigram"]
    n = 0
    # load unigram, bigram, and trigram
    for name in ngram:
        n += 1
        with open( loadPath + "/" + name, "r") as f:
            ngram = {}
            total = 0
            line = f.readline()
            while line:
                kv = line.split( ' ' )
                if len( kv ) > 1:
                    kv[0] = line[:n]
                    kv[1] = line[n + 1:]
                    ngram[kv[0]] = int( kv[1] )
                else:
                    total = int( kv[0] )
                line = f.readline()
            lm[name] = {"c": ngram, "t": total}
    return lm

def addK( lm, s ):
    if s not in lm["trigram"]["c"]:
        cnt1 = 0
    else:
        cnt1 = lm["trigram"]["c"][s]
    if s[:2] not in lm["bigram"]["c"]:
        cnt2 = 0
    else:
        cnt2 = lm["bigram"]["c"][s[:2]]
    p = ( cnt1 + 0.1 ) / ( cnt2 + len( lm["bigram"]["c"] ) * 0.1 ) # hard coded the value 0.1
    return p

def addKSmoothing():
    lm = loadLanguageModel( "./sample_data")
    lambdas = {3: 0.1} # can directly call from here but I habe hard code 0.1 smoothing factor in the addK function.
    TrigramPairs = {}
    for i in range(0,len(testFinal)):
        content = preprocess( testFinal[i] )
        ppw = perplexity( content, lm, func = "AddK", lambdas = lambdas )
        TrigramPairs[test_files[i]] = ppw
    fps = sorted( TrigramPairs.items(), key = lambda x: x[1], reverse = True )

    for fp in fps:
      print("File NAME: "+fp[0]+"--")
      print("Perplexity :"+str(fp[1]))


In [0]:
def part1_3():
    addKSmoothing()

In [79]:
part1_3()

File NAME: ag94se06.txt--
Perplexity :55.2439384292612
File NAME: br94ma01.txt--
Perplexity :54.170750310756276
File NAME: br94ag01.txt--
Perplexity :54.021230858094675
File NAME: ag94ag02.txt--
Perplexity :51.77165232027949
File NAME: ag94ou04.txt--
Perplexity :51.39789717222482
File NAME: br94de01.txt--
Perplexity :51.293150415132054
File NAME: br94jl01.txt--
Perplexity :51.165683354914435
File NAME: ag94de06.txt--
Perplexity :51.15651790024896
File NAME: ag94ju07.txt--
Perplexity :51.125595337785604
File NAME: br94fe1.txt--
Perplexity :50.8167564261631
File NAME: ag94ma03.txt--
Perplexity :50.308019848945484
File NAME: br94ju01.txt--
Perplexity :50.2192854881412
File NAME: ag94ab12.txt--
Perplexity :50.0850870789894
File NAME: br94ja04.txt--
Perplexity :49.675519797893536
File NAME: br94ab02.txt--
Perplexity :49.25760641100549
File NAME: ag94jl12.txt--
Perplexity :49.223728043314225
File NAME: ag94no01.txt--
Perplexity :49.081106606346296
File NAME: ag94mr1.txt--
Perplexity :48.6467

Here the Threshold lies somewhere between 40.88422938855182 - 28.557112434096712 ; Any perplexity value above 40.88422938855182 (included) is non english value. perplexity value 28.557112434096712 (included) and below are english documents.

## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

According to any direct observation we can say by just looking at Thresholds values we can say that Trigram model with add-λ smoothing shows a high difference range of the perplexity values.

The Threshold values of the linear interpolation after we perform the grid serach and choose the best values we get the optimal values but the add-λ smoothing shows better differnec at one go itself.

Linear interpolation is the least sophisticated, least accurate interpolation method. However, it has the advantages that it is fast, and often accurate enough if the table values are closely spaced. Another advantage of this program is that it is easy to find x, given y. This process is sometimes called inverse interpolation.Linear interpolation is quick and easy, but it is not very precise. Another disadvantage is that the interpolant is not differentiable at the point xk.

In the case of add-λ smoothing:
Advantage:Very easy to implement
Disadvantages:Takes too much probability mass from real events, Assigns too much probability to unseen events,Doesn’t take the predicted word into account

Lambda values can change based on the n-gram context.
